In [1]:
# -*- coding: utf-8 -*-
import os
import mwclient
import json
import time
import re
import requests
from bs4 import BeautifulSoup
import lxml
from lxml.html import fromstring

import xlwt
import xlrd
import warnings
warnings.filterwarnings('ignore')

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import pandas as pd

In [2]:
url_nbh = "https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Jerusalem"
webdata_nbh = requests.get(url_nbh)
soup_nbh = BeautifulSoup(webdata_nbh.text, "html.parser")
# neighborhoods chose from 'Pages in category "Neighbourhoods of Jerusalem"'
list_nbh = soup_nbh.find_all('div', attrs={'class':'mw-category-group'})

In [3]:
url_lop = "https://en.wikipedia.org/wiki/List_of_places_in_Jerusalem"
webdata_lop = requests.get(url_lop)
root = fromstring(webdata_lop.content)
table = root.xpath(".//h2[span = 'Neighborhoods']/following-sibling::table")[0]
row = table.xpath(".//tr")[0]
list_lop = [cell.xpath(".//li") for cell in row.xpath(".//td")]

In [4]:
links, titles = [], []
for i in range(12, 34):
    temp_list = list_nbh[i].find_all('li')
    for item in temp_list:
        link = item.find('a')['href']
        link = 'https://en.wikipedia.org/'+str(link)
        title = item.find('a')['title']
        links.append(link)
        titles.append(title)

In [5]:
links2, titles2 = [], []
for sub_list in list_lop:
    for item in sub_list:
        link = item[0].get('href')
        link = 'https://en.wikipedia.org/'+str(link)
        title = item[0].get('title')
        links2.append(link)
        titles2.append(title)

In [6]:
workbook = xlwt.Workbook(r'wiki_nbh_info.xls')
sheet = workbook.add_sheet('wiki_nbh', cell_overwrite_ok=True)
sheet2 = workbook.add_sheet('wiki_lop', cell_overwrite_ok=True)
for i in range(len(links)):
    sheet.write(i, 0, titles[i])
    sheet.write(i, 1, links[i])
    sheet.write(i, 2, 'Category:Neighbourhoods_of_Jerusalem')
for i in range(len(links2)):
    sheet2.write(i, 0, titles2[i])
    sheet2.write(i, 1, links2[i])
    sheet2.write(i, 2, 'List_of_places_in_Jerusalem: Neighborhoods')
workbook.save(r"wiki_nbh_info.xls")

In [7]:
def get_coord(links):
    coord_urls, latitudes, longitudes = [], [], []
    for url in links:   
        webdata = requests.get(url)
        soup = BeautifulSoup(webdata.text, "html.parser")
        try:
            coord = soup.find(id = 'coordinates').findChild('span').findChild('a')
        except:
            coord_url = None
            latitude = None
            longitude = None
        else:
            coord_url = coord['href']
            latitude = coord.findChild('span').findChild('span').findChildren('span')[0].get_text()
            longitude = coord.findChild('span').findChild('span').findChildren('span')[1].get_text()
        coord_urls.append(coord_url)
        latitudes.append(latitude)
        longitudes.append(longitude)
    return coord_urls, latitudes, longitudes

In [8]:
# xls format
# wb_rd = xlrd.open_workbook(r'jerusalem_data.xls')
# sheet_rd = wb_rd.sheet_by_index(0)
# titles3 = sheet_rd.col_values(0, start_rowx=1)

# csv format
book_df = pd.read_csv('jerusalem_data.csv', header=1)
book_df['source_book'] = 1
book_df



,Neighborhood,Year founded,Inhabitants,Initiative,Remarks,Notes,source_book
0,Mishkenot Sha'ananim,1855-1860,"Ashkenazi, Sephardi",Philanthropic,NaN,NaN,1
1,Mahaneh Yisra’el,1866,Maghrebian,Private,NaN,NaN,1
2,Nanalat Shiv’a,1869,"Ashkenazi, Sephardi",Private,NaN,NaN,1
3,Georgian Houses (Eshel Avraham),1872-1876,Georgian,Building society and private,NaN,NaN,1
4,Bet David (Yanover Houses),1873,Ashkenazi,Philanthropic,NaN,NaN,1
...,...,...,...,...,...,...,...
160,Romema,1922,"Ashkenazi and Oriental Jews, Muslims",Private,Named after Psalms 118:16,NaN,1
161,Dajania,Early Mandate,"Muslims, Christians",Private,NaN,*,1
162,al-Musallaba,Early Mandate,Mixed (mainly Jews),NaN,Part of Rehavia?,NaN,1
163,St. Louis Road,Early Mandate,"Jews, Muslims, Christians",NaN,NaN,*,1


In [9]:
nbh_df = pd.DataFrame(data=[titles, links]).T
nbh_df.columns = ['titles','links']
nbh_df['source_NoJ'] = 1
nbh_df

,titles,links,source_NoJ
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1
3,Arnona,https://en.wikipedia.org//wiki/Arnona,1
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1
...,...,...,...
146,Yemin Moshe,https://en.wikipedia.org//wiki/Yemin_Moshe,1
147,Zikhron Moshe,https://en.wikipedia.org//wiki/Zikhron_Moshe,1
148,Zikhron Tuvya,https://en.wikipedia.org//wiki/Zikhron_Tuvya,1
149,Zikhron Yosef,https://en.wikipedia.org//wiki/Zikhron_Yosef,1


In [10]:
lop_df = pd.DataFrame(data=[titles2, links2]).T
lop_df.columns=['titles','links']
lop_df['source_LoP'] = 1
lop_df = lop_df.drop_duplicates()

In [11]:
# merge by the same title and the same link
wiki_df = pd.merge(nbh_df, lop_df, how='outer')
wiki_df

,titles,links,source_NoJ,source_LoP
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1.0,1.0
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1.0,NaN
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1.0,1.0
3,Arnona,https://en.wikipedia.org//wiki/Arnona,1.0,1.0
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1.0,NaN
...,...,...,...,...
174,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0
175,Sanhedria Murchevet,https://en.wikipedia.org//wiki/Sanhedria_Murch...,NaN,1.0
176,Unsdorf,https://en.wikipedia.org//wiki/Unsdorf,NaN,1.0
177,Zichron Moshe,https://en.wikipedia.org//wiki/Zichron_Moshe,NaN,1.0


In [12]:
# wiki_df['coord_urls'], wiki_df['latitudes'], wiki_df['longtitudes'] = get_coord(wiki_df['links'])
# wiki_df

In [13]:
# drop the item with the same link
wiki_df = wiki_df.drop(index = 161)

In [14]:
# same link in the wiki_df
fuzz_wiki_link = pd.DataFrame(data = [process.extract(str(item), wiki_df['links'], limit=3, 
        scorer=fuzz.ratio) for item in wiki_df['links']])

In [15]:
fuzz_wiki_link[['name0', 'score0','index0']] = pd.DataFrame(fuzz_wiki_link[0].tolist(), index=fuzz_wiki_link.index)
fuzz_wiki_link[['name1', 'score1','index1']] = pd.DataFrame(fuzz_wiki_link[1].tolist(), index=fuzz_wiki_link.index)

In [16]:
fuzz_wiki_link = fuzz_wiki_link[fuzz_wiki_link['score1']>96]
fuzz_wiki_link

,0,1,2,name0,score0,index0,name1,score1,index1
6,"(https://en.wikipedia.org//wiki/Bab_az-Zahra, ...","(https://en.wikipedia.org//wiki/Bab_a-Zahara, ...","(https://en.wikipedia.org//wiki/Al-Ram, 88, 108)",https://en.wikipedia.org//wiki/Bab_az-Zahra,100,6,https://en.wikipedia.org//wiki/Bab_a-Zahara,98,152
28,"(https://en.wikipedia.org//wiki/Ein_Karem, 100...","(https://en.wikipedia.org//wiki/Ein_Kerem, 98,...","(https://en.wikipedia.org//wiki/Tel_Arza, 89, ...",https://en.wikipedia.org//wiki/Ein_Karem,100,28,https://en.wikipedia.org//wiki/Ein_Kerem,98,154
40,"(https://en.wikipedia.org//wiki/Givat_Massuah,...","(https://en.wikipedia.org//wiki/Givat_Masua, 9...","(https://en.wikipedia.org//wiki/Givat_Moshe, 9...",https://en.wikipedia.org//wiki/Givat_Massuah,100,40,https://en.wikipedia.org//wiki/Givat_Masua,98,156
65,(https://en.wikipedia.org//wiki/Kiryat_HaYovel...,(https://en.wikipedia.org//wiki/Kiryat_Hayovel...,"(https://en.wikipedia.org//wiki/Kiryat_HaLeom,...",https://en.wikipedia.org//wiki/Kiryat_HaYovel,100,65,https://en.wikipedia.org//wiki/Kiryat_Hayovel,100,162
86,"(https://en.wikipedia.org//wiki/Mea_Shearim, 1...","(https://en.wikipedia.org//wiki/Meah_Shearim, ...","(https://en.wikipedia.org//wiki/Malha, 90, 83)",https://en.wikipedia.org//wiki/Mea_Shearim,100,86,https://en.wikipedia.org//wiki/Meah_Shearim,99,164
97,(https://en.wikipedia.org//wiki/Nahalat_Shiv%2...,"(https://en.wikipedia.org//wiki/Nahalat_Shiva,...",(https://en.wikipedia.org//wiki/Nahalat_Shimon...,https://en.wikipedia.org//wiki/Nahalat_Shiv%27a,100,97,https://en.wikipedia.org//wiki/Nahalat_Shiva,97,167
108,"(https://en.wikipedia.org//wiki/Al-Ram, 100, 108)","(https://en.wikipedia.org//wiki/Ar-Ram, 97, 151)","(https://en.wikipedia.org//wiki/Malha, 93, 83)",https://en.wikipedia.org//wiki/Al-Ram,100,108,https://en.wikipedia.org//wiki/Ar-Ram,97,151
111,"(https://en.wikipedia.org//wiki/Ramat_Eshkol, ...","(https://en.wikipedia.org//wiki/Ramot_Eshkol, ...","(https://en.wikipedia.org//wiki/Ramat_Shlomo, ...",https://en.wikipedia.org//wiki/Ramat_Eshkol,100,111,https://en.wikipedia.org//wiki/Ramot_Eshkol,98,172
125,(https://en.wikipedia.org//wiki/Sanhedria_Murh...,(https://en.wikipedia.org//wiki/Sanhedria_Murc...,"(https://en.wikipedia.org//wiki/Sanhedria, 90,...",https://en.wikipedia.org//wiki/Sanhedria_Murhevet,100,125,https://en.wikipedia.org//wiki/Sanhedria_Murch...,99,175
147,"(https://en.wikipedia.org//wiki/Zikhron_Moshe,...","(https://en.wikipedia.org//wiki/Zichron_Moshe,...","(https://en.wikipedia.org//wiki/Zikhron_Yosef,...",https://en.wikipedia.org//wiki/Zikhron_Moshe,100,147,https://en.wikipedia.org//wiki/Zichron_Moshe,98,177


In [17]:
wiki_df.loc[wiki_df.index.isin(fuzz_wiki_link['index1'])]

,titles,links,source_NoJ,source_LoP
6,Bab az-Zahra,https://en.wikipedia.org//wiki/Bab_az-Zahra,1.0,NaN
28,Ein Karem,https://en.wikipedia.org//wiki/Ein_Karem,1.0,NaN
40,Givat Massuah,https://en.wikipedia.org//wiki/Givat_Massuah,1.0,NaN
86,Mea Shearim,https://en.wikipedia.org//wiki/Mea_Shearim,1.0,NaN
97,Nahalat Shiv'a,https://en.wikipedia.org//wiki/Nahalat_Shiv%27a,1.0,NaN
108,Al-Ram,https://en.wikipedia.org//wiki/Al-Ram,1.0,NaN
111,Ramat Eshkol,https://en.wikipedia.org//wiki/Ramat_Eshkol,1.0,NaN
125,Sanhedria Murhevet,https://en.wikipedia.org//wiki/Sanhedria_Murhevet,1.0,NaN
147,Zikhron Moshe,https://en.wikipedia.org//wiki/Zikhron_Moshe,1.0,NaN
149,Zikhron Yosef,https://en.wikipedia.org//wiki/Zikhron_Yosef,1.0,NaN


In [18]:
wiki_df = wiki_df.drop(index = wiki_df.loc[(wiki_df.index.isin(fuzz_wiki_link['index1'])) & (wiki_df['source_LoP']==1)].index)

In [19]:
wiki_df.loc[wiki_df.index.isin(fuzz_wiki_link['index1']),'source_LoP'] = 1
wiki_df[(wiki_df.index.isin(fuzz_wiki_link['index1']))]

,titles,links,source_NoJ,source_LoP
6,Bab az-Zahra,https://en.wikipedia.org//wiki/Bab_az-Zahra,1.0,1.0
28,Ein Karem,https://en.wikipedia.org//wiki/Ein_Karem,1.0,1.0
40,Givat Massuah,https://en.wikipedia.org//wiki/Givat_Massuah,1.0,1.0
86,Mea Shearim,https://en.wikipedia.org//wiki/Mea_Shearim,1.0,1.0
97,Nahalat Shiv'a,https://en.wikipedia.org//wiki/Nahalat_Shiv%27a,1.0,1.0
108,Al-Ram,https://en.wikipedia.org//wiki/Al-Ram,1.0,1.0
111,Ramat Eshkol,https://en.wikipedia.org//wiki/Ramat_Eshkol,1.0,1.0
125,Sanhedria Murhevet,https://en.wikipedia.org//wiki/Sanhedria_Murhevet,1.0,1.0
147,Zikhron Moshe,https://en.wikipedia.org//wiki/Zikhron_Moshe,1.0,1.0
149,Zikhron Yosef,https://en.wikipedia.org//wiki/Zikhron_Yosef,1.0,1.0


In [20]:
wiki_df = wiki_df.reset_index(drop = True)
wiki_df

,titles,links,source_NoJ,source_LoP
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1.0,1.0
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1.0,NaN
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1.0,1.0
3,Arnona,https://en.wikipedia.org//wiki/Arnona,1.0,1.0
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1.0,NaN
...,...,...,...,...
162,Moroccan Quarter,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0
163,Ramot,https://en.wikipedia.org//wiki/Ramot,NaN,1.0
164,"Romema, Jerusalem","https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0
165,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0


In [21]:
wiki_df['coord_urls'], wiki_df['latitudes'], wiki_df['longitudes'] = get_coord(wiki_df['links'])
wiki_df

,titles,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′49.1″N,35°13′57.5″E
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′24″N,35°13′52″E
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°46′30″N,35°13′46″E
3,Arnona,https://en.wikipedia.org//wiki/Arnona,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′N,35°13′E
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′35.09″N,35°13′25.44″E
...,...,...,...,...,...,...,...
162,Moroccan Quarter,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′36″N,35°14′3″E
163,Ramot,https://en.wikipedia.org//wiki/Ramot,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°48′43″N,35°12′0″E
164,"Romema, Jerusalem","https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°47′24″N,35°12′14″E
165,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=K...,31°45′40″N,35°12′25″E


In [22]:
# translate to google coordinates
# all the coordinates are in North and East
wiki_df['latitudes_g'] = wiki_df['latitudes'].apply(lambda x: (float(x.split('°')[0])+
                                                    (float(x.split('°')[1].split('′')[0])/60  if ('′' in x)  else   0) +
                                                    (float(x.split('°')[1].split('′')[1].split('″')[0])/3600  if ('″' in x)  else   0)) if(x) else '-')

wiki_df['longitudes_g'] = wiki_df['longitudes'].apply(lambda x: (float(x.split('°')[0])+
                                                    (float(x.split('°')[1].split('′')[0])/60  if ('′' in x)  else   0) +
                                                    (float(x.split('°')[1].split('′')[1].split('″')[0])/3600  if ('″' in x)  else   0)) if(x) else '-')
wiki_df

,titles,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes,latitudes_g,longitudes_g
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′49.1″N,35°13′57.5″E,31.763639,35.232639
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′24″N,35°13′52″E,31.79,35.231111
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°46′30″N,35°13′46″E,31.775,35.229444
3,Arnona,https://en.wikipedia.org//wiki/Arnona,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′N,35°13′E,31.75,35.216667
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′35.09″N,35°13′25.44″E,31.793081,35.223733
...,...,...,...,...,...,...,...,...,...
162,Moroccan Quarter,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′36″N,35°14′3″E,31.776667,35.234167
163,Ramot,https://en.wikipedia.org//wiki/Ramot,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°48′43″N,35°12′0″E,31.811944,35.2
164,"Romema, Jerusalem","https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°47′24″N,35°12′14″E,31.79,35.203889
165,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=K...,31°45′40″N,35°12′25″E,31.761111,35.206944


In [23]:
wiki_cor_df = wiki_df[~wiki_df['coord_urls'].isnull()]
wiki_cor_df

,titles,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes,latitudes_g,longitudes_g
0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′49.1″N,35°13′57.5″E,31.763639,35.232639
1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′24″N,35°13′52″E,31.79,35.231111
2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°46′30″N,35°13′46″E,31.775,35.229444
3,Arnona,https://en.wikipedia.org//wiki/Arnona,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′N,35°13′E,31.75,35.216667
4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′35.09″N,35°13′25.44″E,31.793081,35.223733
...,...,...,...,...,...,...,...,...,...
162,Moroccan Quarter,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′36″N,35°14′3″E,31.776667,35.234167
163,Ramot,https://en.wikipedia.org//wiki/Ramot,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°48′43″N,35°12′0″E,31.811944,35.2
164,"Romema, Jerusalem","https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°47′24″N,35°12′14″E,31.79,35.203889
165,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=K...,31°45′40″N,35°12′25″E,31.761111,35.206944


In [24]:
result = wiki_cor_df[['titles','latitudes_g','longitudes_g']].to_json('wiki_cor.json',orient="records")

In [25]:
# find the exactly matched neighborhoods
perfect_match_df = pd.merge(book_df,wiki_cor_df,left_on=['Neighborhood'],right_on=['titles'])
perfect_match_df

,Neighborhood,Year founded,Inhabitants,Initiative,Remarks,Notes,source_book,titles,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes,latitudes_g,longitudes_g
0,Mishkenot Sha'ananim,1855-1860,"Ashkenazi, Sephardi",Philanthropic,NaN,NaN,1,Mishkenot Sha'ananim,https://en.wikipedia.org//wiki/Mishkenot_Sha%2...,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′17.05″N,35°13′27.65″E,31.771403,35.224347
1,Mazkeret Moshe,1882,Ashkenazi,Building society,NaN,NaN,1,Mazkeret Moshe,https://en.wikipedia.org//wiki/Mazkeret_Moshe,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′59″N,35°12′50″E,31.783056,35.213889
2,Zikhron Tuvya,1890,Mixed,Commercial,NaN,NaN,1,Zikhron Tuvya,https://en.wikipedia.org//wiki/Zikhron_Tuvya,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=Z...,31°46′59.93″N,35°12′43.71″E,31.783314,35.212142
3,Yemin Moshe,1892-1894,"Sephardi, Ashkenazi",Building society,NaN,NaN,1,Yemin Moshe,https://en.wikipedia.org//wiki/Yemin_Moshe,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=Y...,31°46′19″N,35°13′30″E,31.771944,35.225
4,Zikhron Moshe,1906,mostly Ashkenazi,Building society,NaN,NaN,1,Zikhron Moshe,https://en.wikipedia.org//wiki/Zikhron_Moshe,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=Z...,31°47′15″N,35°13′00″E,31.7875,35.216667
5,Russian Compound,1858-1864,Russian Orthodox,Philanthropic,NaN,NaN,1,Russian Compound,https://en.wikipedia.org//wiki/Russian_Compound,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°46′55.47″N,35°13′18.17″E,31.782075,35.221714
6,Mamilla,end of 19th century,Mixed,Private,NaN,NaN,1,Mamilla,https://en.wikipedia.org//wiki/Mamilla,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′34.02″N,35°13′24.44″E,31.776117,35.223456
7,Musrara,early 20th century,Christian Arabs,Private,NaN,NaN,1,Musrara,https://en.wikipedia.org//wiki/Musrara,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′57″N,35°13′32″E,31.7825,35.225556
8,Talbiya,early 20th century,Christian Arabs,Private,NaN,NaN,1,Talbiya,https://en.wikipedia.org//wiki/Talbiya,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=T...,31°46′N,35°13′E,31.766667,35.216667
9,Talpiot,1922,Bank clerks and others,Private,Garden-suburb; named after Song of Songs 4:4,NaN,1,Talpiot,https://en.wikipedia.org//wiki/Talpiot,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=T...,31°45′N,35°13′E,31.75,35.216667


In [26]:
# see results
result = perfect_match_df.to_json('perfect_match.json',orient="records")

In [27]:
# left items in book
book_left_df = book_df[~book_df['Neighborhood'].isin(perfect_match_df['Neighborhood'])]
book_left_df = book_left_df.reset_index()
book_left_df

,index,Neighborhood,Year founded,Inhabitants,Initiative,Remarks,Notes,source_book
0,1,Mahaneh Yisra’el,1866,Maghrebian,Private,NaN,NaN,1
1,2,Nanalat Shiv’a,1869,"Ashkenazi, Sephardi",Private,NaN,NaN,1
2,3,Georgian Houses (Eshel Avraham),1872-1876,Georgian,Building society and private,NaN,NaN,1
3,4,Bet David (Yanover Houses),1873,Ashkenazi,Philanthropic,NaN,NaN,1
4,5,Me'ah She'arim,1875,Ashkenazi,Building society,NaN,NaN,1
...,...,...,...,...,...,...,...,...
142,159,Wadi Haluwa,End of Mandate,Muslims,NaN,NaN,NaN,1
143,161,Dajania,Early Mandate,"Muslims, Christians",Private,NaN,*,1
144,162,al-Musallaba,Early Mandate,Mixed (mainly Jews),NaN,Part of Rehavia?,NaN,1
145,163,St. Louis Road,Early Mandate,"Jews, Muslims, Christians",NaN,NaN,*,1


In [28]:
# left items in wiki
wiki_left_df = wiki_cor_df[~wiki_df['titles'].isin(perfect_match_df['titles'])]
wiki_left_df = wiki_left_df.reset_index()
wiki_left_df

,index,titles,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes,latitudes_g,longitudes_g
0,0,Abu Tor,https://en.wikipedia.org//wiki/Abu_Tor,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°45′49.1″N,35°13′57.5″E,31.763639,35.232639
1,1,"American Colony, Jerusalem",https://en.wikipedia.org//wiki/American_Colony...,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′24″N,35°13′52″E,31.79,35.231111
2,2,Armenian Quarter,https://en.wikipedia.org//wiki/Armenian_Quarter,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°46′30″N,35°13′46″E,31.775,35.229444
3,4,Arzei HaBira,https://en.wikipedia.org//wiki/Arzei_HaBira,1.0,NaN,//geohack.toolforge.org/geohack.php?pagename=A...,31°47′35.09″N,35°13′25.44″E,31.793081,35.223733
4,5,Atarot,https://en.wikipedia.org//wiki/Atarot,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=A...,31°51′N,35°13′E,31.85,35.216667
...,...,...,...,...,...,...,...,...,...,...
128,162,Moroccan Quarter,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′36″N,35°14′3″E,31.776667,35.234167
129,163,Ramot,https://en.wikipedia.org//wiki/Ramot,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°48′43″N,35°12′0″E,31.811944,35.2
130,164,"Romema, Jerusalem","https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°47′24″N,35°12′14″E,31.79,35.203889
131,165,"San Simon, Jerusalem","https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=K...,31°45′40″N,35°12′25″E,31.761111,35.206944


In [29]:
# match the left data

fuzz_match_df = pd.concat([book_left_df['Neighborhood'],pd.DataFrame(data = [process.extract(str(item), wiki_left_df['titles'], limit=3, 
        scorer=fuzz.ratio) for item in book_left_df['Neighborhood']], columns=['sub1','sub2','sub3'])],axis= 1)
fuzz_match_df

,Neighborhood,sub1,sub2,sub3
0,Mahaneh Yisra’el,"(Mahane Israel, 90, 71)","(Knesset Yisrael, 65, 67)","(Beit Yisrael, 64, 19)"
1,Nanalat Shiv’a,"(Nahalat Shiv'a, 93, 83)","(Nahalat Shimon, 71, 82)","(Ramat Sharett, 52, 98)"
2,Georgian Houses (Eshel Avraham),"(Givat HaVradim, 45, 121)","(Mea Shearim, 44, 74)","(German Colony, Jerusalem, 41, 31)"
3,Bet David (Yanover Houses),"(Beit Ya'akov, Jerusalem, 54, 18)","(Beit David, 51, 13)","(Beit HaKerem, Jerusalem, 50, 14)"
4,Me'ah She'arim,"(Mea Shearim, 88, 74)","(Ramat Sharett, 52, 98)","(Nahalat Shimon, 50, 82)"
...,...,...,...,...
142,Wadi Haluwa,"(Wadi Hilweh, 73, 117)","(Wadi al-Joz, 64, 116)","(Arzei HaBira, 52, 3)"
143,Dajania,"(Mahane Israel, 50, 71)","(Beit Hanina, 44, 15)","(Al-Issawiya, 44, 123)"
144,al-Musallaba,"(Al-Walaja, 67, 118)","(Jabel Mukaber, 56, 49)","(Al-Issawiya, 52, 123)"
145,St. Louis Road,"(Knesset Yisrael, 48, 67)","(Beit Yisrael, 46, 19)","(Ezrat Yisrael, 44, 29)"


In [30]:
fuzz_match_df[['name1', 'score1','index1']] = pd.DataFrame(fuzz_match_df['sub1'].tolist(), index=fuzz_match_df.index)
fuzz_match_df[['name2', 'score2','index2']] = pd.DataFrame(fuzz_match_df['sub2'].tolist(), index=fuzz_match_df.index)
fuzz_match_df[['name3', 'score3','index3']] = pd.DataFrame(fuzz_match_df['sub3'].tolist(), index=fuzz_match_df.index)

In [31]:
# find scores more than 70
fuzz_match_df[(fuzz_match_df['score1']>70)|(fuzz_match_df['score2']>70)|(fuzz_match_df['score3']>70)]

,Neighborhood,sub1,sub2,sub3,name1,score1,index1,name2,score2,index2,name3,score3,index3
0,Mahaneh Yisra’el,"(Mahane Israel, 90, 71)","(Knesset Yisrael, 65, 67)","(Beit Yisrael, 64, 19)",Mahane Israel,90,71,Knesset Yisrael,65,67,Beit Yisrael,64,19
1,Nanalat Shiv’a,"(Nahalat Shiv'a, 93, 83)","(Nahalat Shimon, 71, 82)","(Ramat Sharett, 52, 98)",Nahalat Shiv'a,93,83,Nahalat Shimon,71,82,Ramat Sharett,52,98
4,Me'ah She'arim,"(Mea Shearim, 88, 74)","(Ramat Sharett, 52, 98)","(Nahalat Shimon, 50, 82)",Mea Shearim,88,74,Ramat Sharett,52,98,Nahalat Shimon,50,82
5,Even Yisra’el,"(Beit Yisrael, 72, 19)","(Knesset Yisrael, 71, 67)","(Ezrat Yisrael, 69, 29)",Beit Yisrael,72,19,Knesset Yisrael,71,67,Ezrat Yisrael,69,29
9,Bet Ya'akov,"(Neve Yaakov, 73, 87)","(Beit Ya'akov, Jerusalem, 65, 18)","(Beit David, 57, 13)",Neve Yaakov,73,87,"Beit Ya'akov, Jerusalem",65,18,Beit David,57,13
14,(Old) Bet Yisra'el,"(Beit Yisrael, 76, 19)","(Ezrat Yisrael, 67, 29)","(Knesset Yisrael, 62, 67)",Beit Yisrael,76,19,Ezrat Yisrael,67,29,Knesset Yisrael,62,67
20,Sha'arei Tzedeq,"(Sha'arei Hesed, 76, 105)","(Sha'arei Yerushalayim, 56, 106)","(Har Hazeitim, 52, 124)",Sha'arei Hesed,76,105,Sha'arei Yerushalayim,56,106,Har Hazeitim,52,124
24,Shim'on haTzadiq Houses,"(Shimon HaTzadik, 74, 111)","(Shikun Chabad, 50, 110)","(San Simon, Jerusalem, 47, 131)",Shimon HaTzadik,74,111,Shikun Chabad,50,110,"San Simon, Jerusalem",47,131
26,Sha'arei Yerushalayim (“Abu-Bassal”),"(Sha'arei Yerushalayim, 76, 106)","(Jewish Quarter (Jerusalem), 47, 53)","(Sha'arei Hesed, 46, 105)",Sha'arei Yerushalayim,76,106,Jewish Quarter (Jerusalem),47,53,Sha'arei Hesed,46,105
27,‘Ezrat Yisra'el,"(Ezrat Yisrael, 96, 29)","(Beit Yisrael, 77, 19)","(Knesset Yisrael, 69, 67)",Ezrat Yisrael,96,29,Beit Yisrael,77,19,Knesset Yisrael,69,67


In [32]:
fuzz_match_df

,Neighborhood,sub1,sub2,sub3,name1,score1,index1,name2,score2,index2,name3,score3,index3
0,Mahaneh Yisra’el,"(Mahane Israel, 90, 71)","(Knesset Yisrael, 65, 67)","(Beit Yisrael, 64, 19)",Mahane Israel,90,71,Knesset Yisrael,65,67,Beit Yisrael,64,19
1,Nanalat Shiv’a,"(Nahalat Shiv'a, 93, 83)","(Nahalat Shimon, 71, 82)","(Ramat Sharett, 52, 98)",Nahalat Shiv'a,93,83,Nahalat Shimon,71,82,Ramat Sharett,52,98
2,Georgian Houses (Eshel Avraham),"(Givat HaVradim, 45, 121)","(Mea Shearim, 44, 74)","(German Colony, Jerusalem, 41, 31)",Givat HaVradim,45,121,Mea Shearim,44,74,"German Colony, Jerusalem",41,31
3,Bet David (Yanover Houses),"(Beit Ya'akov, Jerusalem, 54, 18)","(Beit David, 51, 13)","(Beit HaKerem, Jerusalem, 50, 14)","Beit Ya'akov, Jerusalem",54,18,Beit David,51,13,"Beit HaKerem, Jerusalem",50,14
4,Me'ah She'arim,"(Mea Shearim, 88, 74)","(Ramat Sharett, 52, 98)","(Nahalat Shimon, 50, 82)",Mea Shearim,88,74,Ramat Sharett,52,98,Nahalat Shimon,50,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Wadi Haluwa,"(Wadi Hilweh, 73, 117)","(Wadi al-Joz, 64, 116)","(Arzei HaBira, 52, 3)",Wadi Hilweh,73,117,Wadi al-Joz,64,116,Arzei HaBira,52,3
143,Dajania,"(Mahane Israel, 50, 71)","(Beit Hanina, 44, 15)","(Al-Issawiya, 44, 123)",Mahane Israel,50,71,Beit Hanina,44,15,Al-Issawiya,44,123
144,al-Musallaba,"(Al-Walaja, 67, 118)","(Jabel Mukaber, 56, 49)","(Al-Issawiya, 52, 123)",Al-Walaja,67,118,Jabel Mukaber,56,49,Al-Issawiya,52,123
145,St. Louis Road,"(Knesset Yisrael, 48, 67)","(Beit Yisrael, 46, 19)","(Ezrat Yisrael, 44, 29)",Knesset Yisrael,48,67,Beit Yisrael,46,19,Ezrat Yisrael,44,29


In [33]:
# fuzz_match_df.loc['partial_score1'] = fuzz_match_df.apply(lambda x:fuzz.partial_ratio(book_left_df['Neighborhood'],x['name1']), axis=0)
# fuzz_match_df
                                 

In [82]:
book_df = book_df[~book_df['Neighborhood'].isnull()]

In [83]:
# whole dataset after perfect match 
wiki_temp_df = wiki_cor_df.rename(columns={'titles':'Neighborhood'})
total_list_permatch = pd.merge(book_df,wiki_temp_df,how = 'outer')
total_list_permatch

,Neighborhood,Year founded,Inhabitants,Initiative,Remarks,Notes,source_book,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes,latitudes_g,longitudes_g
0,Mishkenot Sha'ananim,1855-1860,"Ashkenazi, Sephardi",Philanthropic,NaN,NaN,1.0,https://en.wikipedia.org//wiki/Mishkenot_Sha%2...,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′17.05″N,35°13′27.65″E,31.771403,35.224347
1,Mahaneh Yisra’el,1866,Maghrebian,Private,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nanalat Shiv’a,1869,"Ashkenazi, Sephardi",Private,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Georgian Houses (Eshel Avraham),1872-1876,Georgian,Building society and private,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bet David (Yanover Houses),1873,Ashkenazi,Philanthropic,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Moroccan Quarter,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′36″N,35°14′3″E,31.776667,35.234167
293,Ramot,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org//wiki/Ramot,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°48′43″N,35°12′0″E,31.811944,35.2
294,"Romema, Jerusalem",NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°47′24″N,35°12′14″E,31.79,35.203889
295,"San Simon, Jerusalem",NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=K...,31°45′40″N,35°12′25″E,31.761111,35.206944


In [84]:
total_list_permatch = total_list_permatch.rename(columns={'Year founded':'Year_founded'})
total_list_permatch

,Neighborhood,Year_founded,Inhabitants,Initiative,Remarks,Notes,source_book,links,source_NoJ,source_LoP,coord_urls,latitudes,longitudes,latitudes_g,longitudes_g
0,Mishkenot Sha'ananim,1855-1860,"Ashkenazi, Sephardi",Philanthropic,NaN,NaN,1.0,https://en.wikipedia.org//wiki/Mishkenot_Sha%2...,1.0,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′17.05″N,35°13′27.65″E,31.771403,35.224347
1,Mahaneh Yisra’el,1866,Maghrebian,Private,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nanalat Shiv’a,1869,"Ashkenazi, Sephardi",Private,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Georgian Houses (Eshel Avraham),1872-1876,Georgian,Building society and private,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bet David (Yanover Houses),1873,Ashkenazi,Philanthropic,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Moroccan Quarter,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org//wiki/Moroccan_Quarter,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=M...,31°46′36″N,35°14′3″E,31.776667,35.234167
293,Ramot,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org//wiki/Ramot,NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°48′43″N,35°12′0″E,31.811944,35.2
294,"Romema, Jerusalem",NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org//wiki/Romema,_Jerusalem",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=R...,31°47′24″N,35°12′14″E,31.79,35.203889
295,"San Simon, Jerusalem",NaN,NaN,NaN,NaN,NaN,NaN,"https://en.wikipedia.org//wiki/San_Simon,_Jeru...",NaN,1.0,//geohack.toolforge.org/geohack.php?pagename=K...,31°45′40″N,35°12′25″E,31.761111,35.206944


In [85]:
total_list_permatch.to_csv('./total_list_permatch.csv',index = False)

In [89]:
total_list_permatch = pd.read_csv('./total_list_permatch.csv')
total_list_permatch.to_json(orient = 'records')

'[{"Neighborhood":"Mishkenot Sha\'ananim","Year_founded":"1855-1860","Inhabitants":"Ashkenazi, Sephardi","Initiative":"Philanthropic","Remarks":null,"Notes":null,"source_book":1.0,"links":"https:\\/\\/en.wikipedia.org\\/\\/wiki\\/Mishkenot_Sha%27ananim","source_NoJ":1.0,"source_LoP":1.0,"coord_urls":"\\/\\/geohack.toolforge.org\\/geohack.php?pagename=Mishkenot_Sha%27ananim&params=31_46_17.05_N_35_13_27.65_E_","latitudes":"31\\u00b046\\u203217.05\\u2033N","longitudes":"35\\u00b013\\u203227.65\\u2033E","latitudes_g":31.7714027778,"longitudes_g":35.2243472222},{"Neighborhood":"Mahaneh Yisra\\u2019el","Year_founded":"1866","Inhabitants":"Maghrebian","Initiative":"Private","Remarks":null,"Notes":null,"source_book":1.0,"links":null,"source_NoJ":null,"source_LoP":null,"coord_urls":null,"latitudes":null,"longitudes":null,"latitudes_g":null,"longitudes_g":null},{"Neighborhood":"Nanalat Shiv\\u2019a","Year_founded":"1869","Inhabitants":"Ashkenazi, Sephardi","Initiative":"Private","Remarks":null,